Step 1/2:

Imports libraries for use

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
enc = LabelEncoder()
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('books.csv', on_bad_lines='warn')
df

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco


In [3]:
df.head(2).transpose()

,0,1
bookID,1,2
title,Harry Potter and the Half-Blood Prince (Harry ...,Harry Potter and the Order of the Phoenix (Har...
authors,J.K. Rowling/Mary GrandPré,J.K. Rowling/Mary GrandPré
average_rating,4.57,4.49
isbn,0439785960,0439358078
isbn13,9780439785969,9780439358071
language_code,eng,eng
num_pages,652,870
ratings_count,2095690,2153167
text_reviews_count,27591,29221


Step 3:

Popularity Recommender

In [4]:
def popularityRecommender(df):
    
    #Define the minimum vote count
    minimum_ratings_count = 0.75* df['ratings_count'].max()
    
    #Define C – the mean rating
    mean_rating = df['average_rating'].mean()

    df['weighted_rating'] = (((df['ratings_count']/(df['ratings_count']+minimum_ratings_count)) * df['average_rating']) +
                             ((minimum_ratings_count/(df['ratings_count']+minimum_ratings_count))*mean_rating))

    recommendations = df.sort_values(by = 'weighted_rating',ascending = False).head(5)
    
    return(recommendations)


In [5]:
popularityRecommender(df)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,weighted_rating
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,4.187089
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,4.174464
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,4.147771
4415,15881,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.42,0439064864,9780439064866,eng,341,2293963,34692,6/2/1999,Arthur A. Levine Books / Scholastic Inc.,4.128198
23,34,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,4.36,0618346252,9780618346257,eng,398,2128944,13670,9/5/2003,Houghton Mifflin Harcourt,4.096661


Step 4:

Content Based Recommender (It was recommended to use authors in the TFIDF, but as you could only find the same author using the author column, I decided it would be more optimal to use the title column to find books with similarly named books as a reccomendation instead)

In [42]:
def ContentBasedRecommender(title, indices, distance_matrix):
    id_ = indices[title]
    distances = list(enumerate(distance_matrix[id_]))
    distances = sorted(distances, key=lambda x: x[1], reverse = True)
    distances = distances[1:11]
    recommendations = [i[0] for i in distances]
    return df['title'].iloc[recommendations]

def main():
    cbr = TfidfVectorizer(stop_words='english')
    df['title'] = df['title'].fillna('')
    
    tfidf_matrix = cbr.fit_transform(df['title'])
    distance_matrix = cosine_similarity(tfidf_matrix)
    
    
       

    indices = pd.Series(df.index, index=df['title']).drop_duplicates()
   
    return ContentBasedRecommender('Poor People', indices,distance_matrix)

In [43]:
main()

7708                              The Book of Other People
3012                The Working Poor: Invisible in America
3435                                        All New People
9510                                   A Man of the People
217                          We Were Not Like Other People
6735                                    The Rainbow People
10692                                  The People of Paper
10079                Europe and the People Without History
4011                                    Independent People
2772     Pathologies of Power: Health  Human Rights and...
Name: title, dtype: object